In [1]:
import bitdotio
import os
import pandas as pd

from dotenv import load_dotenv
from sqlalchemy import create_engine

In [2]:
load_dotenv()
os.getenv("BITIO_USER")

BITIO_REPO = "cop26_methane"

# Methane Emissions by Sector
Source: https://cfpub.epa.gov/ghgdata/inventoryexplorer/#iallsectors/allsectors/methane/inventsect/all
"Where does methane come from"

In [9]:
df = pd.read_csv("../data/epa_methane_by_sector.csv")

# convert wide to long
df = pd.melt(df, id_vars=["Emissions of Methane, MMT CO2 eq."], var_name="Year", value_name="mmt_co2_eq")
df.rename(columns={"Year": "year", "Emissions of Methane, MMT CO2 eq.": "sector"}, inplace=True)

engine = create_engine(os.getenv("PG_STRING"))

with engine.connect() as conn:
    df.to_sql(
    "epa_methane_sources",
    conn,
    schema=f'{os.getenv("BITIO_USER")}/{BITIO_REPO}',
    index=False,
    if_exists="replace")

# GWP of Methane
"Why do we care"
Source: https://www.ipcc.ch/report/ar5/wg1/

In [11]:
# get GWPs from HFC repo
engine = create_engine(os.getenv("PG_STRING"))
# SQL for querying an entire table
sql = f'''
    SELECT *
    FROM "bitdotio/hfc"."global_warming_potentials";
'''
# Return SQL query as a pandas dataframe
with engine.connect() as conn:
    # Set 1 minute statement timeout (units are milliseconds)
    conn.execute("SET statement_timeout = 60000;")
    df = pd.read_sql(sql, conn)

    df.to_sql(
    "ipcc_global_warming_potentials",
    conn,
    schema=f'{os.getenv("BITIO_USER")}/{BITIO_REPO}',
    index=False,
    if_exists="replace")

# International Comparison

In [14]:
unfccc_inventory = pd.read_csv("../data/processed_comparison.csv")
engine = create_engine(os.getenv("PG_STRING"))

with engine.connect() as conn:
    unfccc_inventory.to_sql(
    "unfccc_inventory",
    conn,
    schema=f'{os.getenv("BITIO_USER")}/{BITIO_REPO}',
    index=False,
    if_exists="replace")